# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/ex8/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [9]:
pd.DataFrame(ext_data['train']['features'])

,0,1,2,3,4,5,6,7,8,9,...,3766,3767,3768,3769,3770,3771,3772,3773,3774,3775
0,0.243716,0.338328,0.272088,0.226127,0.029367,0.339973,0.209255,0.105823,0.137297,0.145012,...,0.022174,0.006773,0.069666,0.301964,0.180498,0.033291,0.098398,0.024299,0.016897,0.123153
1,0.237665,0.327515,0.263212,0.232293,0.027842,0.367579,0.210006,0.094775,0.134482,0.143913,...,0.027657,0.028354,0.165744,0.330388,0.136494,0.035396,0.146613,0.123391,0.007483,0.144597
2,0.225514,0.379070,0.276066,0.196089,0.027455,0.265828,0.224954,0.132161,0.143215,0.143278,...,0.040949,0.034133,0.122343,0.213779,0.166338,0.049096,0.042672,0.044685,0.002102,0.145841
3,0.241087,0.328841,0.287649,0.230211,0.029166,0.339409,0.215851,0.104063,0.135149,0.140004,...,0.047530,0.000834,0.029579,0.157189,0.159237,0.092072,0.176285,0.072260,0.025339,0.222447
4,0.247998,0.328309,0.278905,0.237095,0.027144,0.362545,0.209568,0.100579,0.133083,0.137760,...,0.022089,0.001686,0.082939,0.303310,0.165293,0.078591,0.169059,0.093608,0.015137,0.118250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6013,0.242294,0.320149,0.291148,0.246376,0.024519,0.336719,0.213888,0.098774,0.152700,0.135828,...,0.028749,0.020032,0.089271,0.324535,0.164114,0.011856,0.131130,0.048871,0.016199,0.125525
6014,0.239820,0.328761,0.283221,0.232255,0.026091,0.329097,0.217799,0.107723,0.151887,0.142054,...,0.065597,0.009168,0.159468,0.321171,0.235521,0.020378,0.059581,0.037248,0.034675,0.046954
6015,0.233884,0.312639,0.287863,0.243157,0.024677,0.309460,0.243808,0.103488,0.180332,0.135159,...,0.159631,0.007203,0.071548,0.284185,0.112961,0.240836,0.032650,0.014332,0.023457,0.157205
6016,0.232720,0.344319,0.275893,0.233720,0.027888,0.292808,0.227276,0.124664,0.170116,0.145821,...,0.098789,0.002348,0.054095,0.253814,0.089476,0.159530,0.034791,0.019259,0.031108,0.169736


## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [11]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [12]:
for machine_type in machine_types:
    run(machine_type)

2021-05-26 22:19:22,910 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-05-26 22:19:22,921 - 00_train.py - INFO - MAKE DATA_LOADER
2021-05-26 22:19:22,975 - 00_train.py - INFO - TRAINING
2021-05-26 22:19:26,928 - pytorch_modeler.py - INFO - train
  0%|          | 0/12 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 12/12 [00:01<00:00,  9.91it/s]
2021-05-26 22:19:28,142 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 92.65it/s]
2021-05-26 22:19:28,189 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 69.72it/s]


,AUC,pAUC
Source_0,0.525800,0.522632
Source_1,0.784000,0.600000
Source_2,0.583000,0.593684
Target_0,0.380700,0.509474
Target_1,0.946600,0.860000
Target_2,0.321700,0.473684
mean,0.590300,0.593246
h_mean,0.513511,0.571365


2021-05-26 22:19:29,263 - pytorch_modeler.py - INFO - epoch:1/300, train_losses:46761.339193, val_AUC_hmean:0.513511, val_pAUC_hmean:0.571365, best_flag:True
2021-05-26 22:19:29,264 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 15.78it/s]
2021-05-26 22:19:30,026 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 60.92it/s]
2021-05-26 22:19:30,085 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 73.33it/s]


...

100%|██████████| 2/2 [00:00<00:00, 59.01it/s]
2021-05-26 22:51:36,136 - pytorch_modeler.py - INFO - epoch:260/300, train_losses:338.567062, val_AUC_hmean:0.647612, val_pAUC_hmean:0.585934, best_flag:False
2021-05-26 22:51:36,137 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 16.17it/s]
2021-05-26 22:51:36,881 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 55.57it/s]
2021-05-26 22:51:36,938 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 62.60it/s]
2021-05-26 22:51:36,990 - pytorch_modeler.py - INFO - epoch:261/300, train_losses:257.633625, val_AUC_hmean:0.671829, val_pAUC_hmean:0.570512, best_flag:False
2021-05-26 22:51:36,991 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 15.98it/s]
2021-05-26 22:51:37,745 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 34.16it/s]
2021-05-26 22:51:37,822 - pytorch_modeler.py - INFO - valid_target
100%|█████████

100%|██████████| 12/12 [00:00<00:00, 16.05it/s]
2021-05-26 22:51:52,881 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 57.44it/s]
2021-05-26 22:51:52,939 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 69.09it/s]
2021-05-26 22:51:52,987 - pytorch_modeler.py - INFO - epoch:277/300, train_losses:97.687263, val_AUC_hmean:0.681117, val_pAUC_hmean:0.592173, best_flag:False
2021-05-26 22:51:52,988 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 16.27it/s]
2021-05-26 22:51:53,728 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 58.44it/s]
2021-05-26 22:51:53,791 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 39.32it/s]
2021-05-26 22:51:53,862 - pytorch_modeler.py - INFO - epoch:278/300, train_losses:106.751427, val_AUC_hmean:0.676329, val_pAUC_hmean:0.593049, best_flag:False
2021-05-26 22:51:53,863 - pytorch_modeler.py - INFO - train
100%|██████████

2021-05-26 22:52:09,530 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:00<00:00, 14.29it/s]
2021-05-26 22:52:10,372 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 60.95it/s]
2021-05-26 22:52:10,428 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 49.39it/s]
2021-05-26 22:52:10,502 - pytorch_modeler.py - INFO - epoch:294/300, train_losses:70.421266, val_AUC_hmean:0.682599, val_pAUC_hmean:0.596005, best_flag:False
2021-05-26 22:52:10,504 - pytorch_modeler.py - INFO - train
100%|██████████| 12/12 [00:01<00:00, 11.09it/s]
2021-05-26 22:52:11,590 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 2/2 [00:00<00:00, 50.81it/s]
2021-05-26 22:52:11,653 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 2/2 [00:00<00:00, 62.54it/s]
2021-05-26 22:52:11,715 - pytorch_modeler.py - INFO - epoch:295/300, train_losses:66.230739, val_AUC_hmean:0.682118, val_pAUC_hmean:0.594123, best_flag:False
2021-05-26 22:52

,AUC,pAUC
Source_0,0.798800,0.651053
Source_1,0.900700,0.783158
Source_2,0.836720,0.813772
Target_0,0.491500,0.493684
Target_1,0.664388,0.573362
Target_2,0.605056,0.498472
mean,0.716194,0.635584
h_mean,0.685735,0.611153


elapsed time: 332.926306009 [sec]


In [13]:
#run(machine_types[0])